In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import resnext50_32x4d
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'resnext32x4d' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 300
start_epoch = 0
train_batch = 400
test_batch = 400
lr = 0.04
schedule = [75, 175, 250]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/32x4d/aug' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
model = resnext50_32x4d(pretrained=False, num_classes=2)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [11]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 22.98M


# Loss

In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Train AUROC.', 'Valid AUROC.'])

# Train

In [14]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        arc.update(auroc, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        if batch_idx % 100 == 0:
            print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [15]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
         batch=batch_idx+1, size=len(val_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [16]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, train_auroc, test_auroc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 300] LR: 0.040000
1/687 | Loss:0.7154 | top1:47.7500 | AUROC:0.4702
101/687 | Loss:4.1193 | top1:50.0050 | AUROC:0.5094
201/687 | Loss:2.4154 | top1:50.4527 | AUROC:0.5359
301/687 | Loss:1.8397 | top1:52.3032 | AUROC:0.5573
401/687 | Loss:1.5492 | top1:53.6758 | AUROC:0.5726
501/687 | Loss:1.3732 | top1:54.7725 | AUROC:0.5855
601/687 | Loss:1.2546 | top1:55.7163 | AUROC:0.5967
687/687 | Loss:1.1795 | top1:56.4548 | AUROC:0.6071
77/77 | Loss:0.6391 | top1:62.9915 | AUROC:0.6866

Epoch: [2 | 300] LR: 0.068000
1/687 | Loss:0.6269 | top1:65.5000 | AUROC:0.7172
101/687 | Loss:0.6502 | top1:61.4257 | AUROC:0.6915
201/687 | Loss:0.6415 | top1:62.6940 | AUROC:0.7039
301/687 | Loss:0.6317 | top1:63.9252 | AUROC:0.7164
401/687 | Loss:0.6185 | top1:65.2070 | AUROC:0.7311
501/687 | Loss:0.6092 | top1:66.1387 | AUROC:0.7437
601/687 | Loss:0.5979 | top1:67.0986 | AUROC:0.7552
687/687 | Loss:0.5882 | top1:67.9042 | AUROC:0.7652
77/77 | Loss:0.4689 | top1:77.3263 | AUROC:0.8616

Epoch: [3

501/687 | Loss:0.0890 | top1:96.5818 | AUROC:0.9964
601/687 | Loss:0.0903 | top1:96.5308 | AUROC:0.9964
687/687 | Loss:0.0910 | top1:96.5102 | AUROC:0.9964
77/77 | Loss:0.0662 | top1:97.4017 | AUROC:0.9979

Epoch: [18 | 300] LR: 0.319684
1/687 | Loss:0.0684 | top1:96.5000 | AUROC:0.9983
101/687 | Loss:0.0906 | top1:96.5990 | AUROC:0.9965
201/687 | Loss:0.0848 | top1:96.7761 | AUROC:0.9968
301/687 | Loss:0.0854 | top1:96.7683 | AUROC:0.9967
401/687 | Loss:0.0843 | top1:96.8055 | AUROC:0.9967
501/687 | Loss:0.0878 | top1:96.6712 | AUROC:0.9965
601/687 | Loss:0.0895 | top1:96.5936 | AUROC:0.9965
687/687 | Loss:0.0904 | top1:96.5722 | AUROC:0.9964
77/77 | Loss:0.0533 | top1:98.0177 | AUROC:0.9985

Epoch: [19 | 300] LR: 0.319570
1/687 | Loss:0.0783 | top1:97.5000 | AUROC:0.9962
101/687 | Loss:0.0830 | top1:96.8465 | AUROC:0.9970
201/687 | Loss:0.0819 | top1:96.9055 | AUROC:0.9972
301/687 | Loss:0.0846 | top1:96.8032 | AUROC:0.9969
401/687 | Loss:0.0852 | top1:96.7880 | AUROC:0.9969
501/687 

101/687 | Loss:0.0573 | top1:97.8688 | AUROC:0.9987
201/687 | Loss:0.0634 | top1:97.6381 | AUROC:0.9983
301/687 | Loss:0.0698 | top1:97.3796 | AUROC:0.9981
401/687 | Loss:0.0687 | top1:97.4059 | AUROC:0.9981
501/687 | Loss:0.0692 | top1:97.3822 | AUROC:0.9980
601/687 | Loss:0.0696 | top1:97.3694 | AUROC:0.9980
687/687 | Loss:0.0715 | top1:97.2952 | AUROC:0.9979
77/77 | Loss:0.0472 | top1:98.2208 | AUROC:0.9989

Epoch: [35 | 300] LR: 0.315381
1/687 | Loss:0.0540 | top1:98.5000 | AUROC:0.9993
101/687 | Loss:0.0780 | top1:97.0099 | AUROC:0.9975
201/687 | Loss:0.0720 | top1:97.2699 | AUROC:0.9977
301/687 | Loss:0.0676 | top1:97.4103 | AUROC:0.9979
401/687 | Loss:0.0686 | top1:97.3809 | AUROC:0.9979
501/687 | Loss:0.0713 | top1:97.2710 | AUROC:0.9979
601/687 | Loss:0.0696 | top1:97.3486 | AUROC:0.9979
687/687 | Loss:0.0707 | top1:97.2912 | AUROC:0.9980
77/77 | Loss:0.0444 | top1:98.2733 | AUROC:0.9991

Epoch: [36 | 300] LR: 0.314973
1/687 | Loss:0.0754 | top1:97.2500 | AUROC:0.9979
101/687 

601/687 | Loss:0.0613 | top1:97.7234 | AUROC:0.9986
687/687 | Loss:0.0614 | top1:97.7099 | AUROC:0.9985
77/77 | Loss:0.0379 | top1:98.5616 | AUROC:0.9995

Epoch: [51 | 300] LR: 0.306841
1/687 | Loss:0.0996 | top1:95.2500 | AUROC:0.9984
101/687 | Loss:0.0517 | top1:98.0866 | AUROC:0.9988
201/687 | Loss:0.0535 | top1:98.0137 | AUROC:0.9987
301/687 | Loss:0.0592 | top1:97.7782 | AUROC:0.9986
401/687 | Loss:0.0578 | top1:97.8398 | AUROC:0.9986
501/687 | Loss:0.0584 | top1:97.8129 | AUROC:0.9986
601/687 | Loss:0.0598 | top1:97.7762 | AUROC:0.9985
687/687 | Loss:0.0591 | top1:97.8043 | AUROC:0.9985
77/77 | Loss:0.0353 | top1:98.6992 | AUROC:0.9994

Epoch: [52 | 300] LR: 0.306167
1/687 | Loss:0.0293 | top1:99.0000 | AUROC:0.9996
101/687 | Loss:0.0613 | top1:97.6262 | AUROC:0.9985
201/687 | Loss:0.0696 | top1:97.3184 | AUROC:0.9982
301/687 | Loss:0.0647 | top1:97.5241 | AUROC:0.9984
401/687 | Loss:0.0628 | top1:97.6166 | AUROC:0.9984
501/687 | Loss:0.0636 | top1:97.5968 | AUROC:0.9984
601/687 